# Анализ ответов на первый вопрос

Импортируем необходимые библиотеки

In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import random
import math
from scipy.stats import shapiro
import scipy.stats as st
import stats_functions as sf
pd.options.mode.chained_assignment = None

Считаем исходные данные из файла csv в датафрейм megafon_df и посмотрим на него

In [ ]:
megafon_df = pd.read_csv('megafon.csv')
megafon_df

Мы видим 3112 абонентов, их ответы на два вопроса и 8 технических параметров связи.

Посмотрим информацию по датафрейму

In [ ]:
megafon_df.info()

Что мы сразу видим:<br>1. В ответе на первый вопрос есть два пропущенных значения<br>2. В ответе на второй вопрос есть много пропущенных значений, но это объяснимо, так как не от всех абонентов требовалось на него отвечать<br>3. Ответы на вопросы подразумевают числовой ответ, а тип столбцов с ответами по факту строковый<br>Разберемся с этими нюансами

Рассмотрим поподробнее числовые столбцы

In [ ]:
megafon_df.describe(include=[np.number])

Мы видим, что значения параметра Total Traffic(MB) распределены более менее равномерно. Остальные же параметры имеют явные выбросы - максимумы значительно больше среднего и медианного значения.

Теперь посмотрим на строковые столбцы

In [ ]:
megafon_df.describe(include=[object])

Мы видим 49 уникальных значений при ответе на первый вопрос, хотя их должно быть десять. Также видим наиболее популярный ответ на первый вопрос - оценка 10, и на второй вопрос - 3, то есть плохое качество связи в зданиях, торговых центрах и т.д.

Давай те посмотрим, что же за 49 вариантов ответов дали абоненты при ответе на первый вопрос

In [ ]:
megafon_df['Q1'].value_counts()

Все понятно - кто то просто дал сразу несколько оценок (видимо, бывает в разных местах и где то связь лучше, а где то хуже). Кто то вообще решил вместо оценок попробовать себя в эпистолярном жанре.

Давайте еще взглянем на двух абонентов, которые не утрудили себя ответом.

In [ ]:
megafon_df[megafon_df['Q1'].isna()]

Итак, почистим данные. Уберем этих двух не пожелавших дать ответ. На второй вопрос они ответа также не дали и смысловой нагрузки для нашего исследования не несут. Далее вручную разберем ответы, отличные от цифр от 1 до 10, и если можно их однозначно интерпретировать в конкретную оценку, сделаем это. Там, где даны два ответа и они близки, возьмем среднюю оценку.

In [ ]:
megafon_df.loc[megafon_df['Q1'] == '1, 3', 'Q1'] = '2'
megafon_df.loc[megafon_df['Q1'] == '5, 6', 'Q1'] = '5'
megafon_df.loc[megafon_df['Q1'] == 'Ужасно', 'Q1'] = '1'
megafon_df.loc[megafon_df['Q1'] == 'Поохое', 'Q1'] = '1'
megafon_df.loc[megafon_df['Q1'] == 'Очень  хорошо. Обслуживания  я довольно. Спасибо вам.555', 'Q1'] = '9'
megafon_df.loc[megafon_df['Q1'] == '3 тройка, связь отвратительная, жалко платить за такой тарив', 'Q1'] = '3'
megafon_df.loc[megafon_df['Q1'] == 'Немагу дать атценку денги незашто снимаеть скоро выклучаю', 'Q1'] = '1'
megafon_df.loc[megafon_df['Q1'] == '4. Тульская область Заокский район. Романовские дачи связи почти нет', 'Q1'] = '4'
megafon_df.loc[megafon_df['Q1'] == '10, 9', 'Q1'] = '10'
megafon_df.loc[megafon_df['Q1'] == 'Чем даль ше,тем лучше.Спасибо за ваш труд.Оценка 10 !', 'Q1'] = '10'
megafon_df.loc[megafon_df['Q1'] == 'ОЦЕНКА-3/НЕВАЖНО/', 'Q1'] = '3'
megafon_df.loc[megafon_df['Q1'] == 'Отвратительно', 'Q1'] = '1'
megafon_df.loc[megafon_df['Q1'] == '5, 7', 'Q1'] = '6'
megafon_df.loc[megafon_df['Q1'] == '5, 7', 'Q1'] = '6'
megafon_df.loc[megafon_df['Q1'] == '0', 'Q1'] = '1'

Итак, данные почищены. Уберем недостоверные и не могущие быть правильно интерпретированными данные в датафрейм undefined_df. (Таких данных очень мало, поэтому безболезненно от них избавляемся.) Зачем оставлять эти данные в отдельном датафрейме? Не знаю, вдруг пригодятся. Не люблю безвозвратно терять данные. Не понадобятся дальше - ну и ладно.

In [ ]:
undefined_df = megafon_df[megafon_df['Q1'].isin(megafon_df['Q1'].value_counts().tail(25).index) | megafon_df['Q1'].isna()]

Посмотрим, сколько же мы выкинули.

In [ ]:
len(undefined_df)

Действительно очень мало.

Теперь создадим датафреймы:<br>1. good_grade_df - сюда войдут все абоненты, поставившие 10 или 9<br>2. bad_grade_df - сюда войдут все остальные абоненты

И сразу же приведем столбец с оценками к целому типу, так как теперь здесь только четкие цифры.

In [ ]:
good_grade_df = megafon_df.loc[((megafon_df['Q1'] == '10') | (megafon_df['Q1'] == '9'))]
good_grade_df = good_grade_df.astype({"Q1": "Int64"})

In [ ]:
bad_grade_df = megafon_df[~megafon_df['user_id'].isin(list(undefined_df['user_id']) + list(good_grade_df['user_id']))]
bad_grade_df = bad_grade_df.astype({"Q1": "Int64"})

Ну и создадим общий почищенный датафрейм.

In [ ]:
defined_df = pd.concat([good_grade_df, bad_grade_df])

Теперь посмотрим на распределение количества оценок при ответе на первый вопрос на базе нашего почищенного датафрейма.

In [ ]:
fig = px.histogram(defined_df, x = 'Q1', range_x = [0, 11], color = 'Q1')
fig.update_traces(marker_line_width=1,marker_line_color="white")
fig.update_layout(title_text="Распределение количества оценок абонентов при ответе на первый вопрос",
                  title_font=dict(size=22, color='blue'))
fig.update_xaxes(nticks=20)
fig.update_xaxes(title_font=dict(size=18, color='green'), title_text='Оценка абонента при ответе на первый вопрос')
fig.update_yaxes(title_font=dict(size=18, color='green'), title_text='Количество поставивших оценку')
fig.show()

Первый вывод, который можно сделать - шкала из 10 оценок довольно размазанная. Легче всего было тем абонентам, которые всем довольны или наоборот которым все не нравится. Это крайние оценки и по ним больше всего ответов. Как оценить например на 4? Почему не 5? И так далее. По этой причине корреляция оценок с техническими показателями скорее всего будет низкая. Давайте это проверим.

Для этого создадим функцию draw_correlation_graphs и упакуем ее в скрипт. (В начале этого блокнота мы ее импортировали). Рассмотрим её работу.<br>На вход функция принимает:<br>- Датафрейм для исследования<br>- Индекс столбца датафрейма, корелляцию с которым мы хотим проверить<br>- Список индексов столбцов, корелляцию которых с целевым надо проверить<br>- Список методов для определения корелляции ('spearman', 'pearson', 'kendall') - по умолчанию 'spearman'

Функция строит сетку графиков, где каждая строка соответстует техническому параметру, корелляцию которого с целевым столбцом надо проверить (индексы столбцов, содержащих эти параметры, мы передаём в функцию), а каждый столбец соответствует методу для определения корелляции (от 1 до 3). Что за графики?

Во-первых, упомянём сами корелляционные методы:<br>1. Пирсон используется для измерения корреляции между двумя непрерывными переменными<br>2. Спирман используется для измерения корреляции между двумя ранжированными переменными<br>3. Кендалл используется, когда вы хотите использовать корреляцию Спирмена, но размер выборки мал и имеется много связанных рангов

У нас целевой параметр - оценка по первому вопросу - это ранжированная переменная. А все технические параметры являются непрерывными переменными, поэтому для правильной корелляционной оценки для методов Спирмана и Кендалла мы должны сделать технический непрерывный параметр ранжированным. Функция делает это сама для этих двух методов с помощью метода pandas.cut - мы делим непрерывную переменную на конечное количество интервалов и каждому интервалу присваиваем номер (ранжируем). Для Пирсона нам надо бы было сделать обратное - перевести ранжированный параметр с оценками в непрерывный формат, но это невозможно, поэтому Пирсон будет вычислять корелляцию между ранжированной и непрерывной переменной, но так как оценок десять - отчасти их можно воспринять как некий условно непрерывный параметр и для оценки корелляции в принципе нам этого будет достаточно.

Следующий нюанс. Как мы видели выше, почти у всех технических параметров есь сильные выбросы вверх, которые могут влиять на коррелляцию. Давайте проверим - если мы отрежем хвост с выбросами, повлияет ли это на корелляцию оценок с техническими параметрами. Вопрос - сколько отрезать и с какой стороны? Решение простое. Посмотрим на пример. Возьмем, например, один из параметров с выбросами - Downlink Throughput(Kbps), обрежем его с обоих сторон по квантилям 3% с одной стороны и 97% с другой и построим для него гистограмму его распределения. Отметим на ней линию, соответствующую среднему значению параметра и медианному.

In [ ]:
q_low = defined_df["Downlink Throughput(Kbps)"].quantile(0.03)
q_hi  = defined_df["Downlink Throughput(Kbps)"].quantile(0.97)

In [ ]:
df_filtered = defined_df[(defined_df["Downlink Throughput(Kbps)"] < q_hi) &
                         (defined_df["Downlink Throughput(Kbps)"] > q_low)]

In [ ]:
fig = px.histogram(df_filtered, x="Downlink Throughput(Kbps)")
fig.update_traces(marker_line_width=1,marker_line_color="white")
fig.update_layout(title_text="Распределение параметра Downlink Throughput(Kbps)",
                  title_font=dict(size=22, color='blue'))
fig.add_vline(x=df_filtered["Downlink Throughput(Kbps)"].mean(), line_width=1, line_dash='solid', line_color='red')
fig.add_vline(x=df_filtered["Downlink Throughput(Kbps)"].median(), line_width=1, line_dash='dash', line_color='red')
fig.show()

Принцип следующий. Если мы будем отрезать слева или справа от этого графика какое то количество данных, убирая выбросы, то распределение будет становится все более нормальным и разница между медианой и средним будет уменьшатся. На этом принципе наша функция и строит графики. А именно:<br>1. Отрезаем на каждой итерации по 1% данных сначала справа и смотрим, насколько уменьшилась разница между медианой и средним<br>2. Делаем то же самое, но справа<br>3. Проверяем, какой из отрезов (справа или слева) привёл к большему уменьшению указанной разницы и с той стороны его уже окончательно обрезаем<br>4. После каждого такого отреза считаем корелляцию и сохраняем ее в список<br>5. Для каждого сочетания целевой переменной, метода для определения корелляции и технического параметра, с которым кореллируем целевую переменную получаем такой список и строим график изменения корелляции при постепенном отрезании выбросов у соответствующего технического параметра.

В функцию можно передавать выборочные тех.параметры связи, но давайте посмотрим на все сразу, чтобы понять, есть ли корелляция оценки абонентов при ответе на перввый вопрос хоть с одним из них и влияет ли убирание выбросов на величину коэффициента корелляции.

Дополнительно на каждый график для удобства визуальной оценки нанесены вертикальные красные прерывистые линии, соответствующие отрезанию соответственно 10% от общего объема значений параметра, 20%, 30%, 40% и 50%. Графики построены до момента полного исчерпания данных просто для красоты. По факту дальше 50%-ной линии смотреть уже нет смысла, так как при этом мы теряем уже более половины данных. Нам нужны такие графики только для того, чтобы понять - есть ли хоть какая то вероятность возникновения значимой корелляции между оценкой абонента и значением тех. параметра.

In [ ]:
sf.draw_correlation_graphs(defined_df,
                           target_feature_idx = 1,
                           columns_idxs = list(range(3, 11)),
                           meth_list = ['pearson', 'spearman', 'kendall'])

Что же мы видим?<br>Ни при каких условиях ни у одного из параметров корелляция не превышает 0.15, что крайне низко и говорит нам о том, что прямой связи роста значения оценки с ростом или падением тех.параметра связи нет. Давайте для успокоения попробуем еще такой ход - создадим дополнительный столбец с условным общим параметром качества связи. То есть рассмотрим не каждый параметр в отдельности, а их совместное влияние на оценку абонента. Для этого все параметры для каждого абонента, значение которых лучше больше, складываем между собой, а те, значение которых лучше меньше, вычитаем.

In [ ]:
defined_df['total_quality'] = defined_df['Downlink Throughput(Kbps)'] + \
                              defined_df['Uplink Throughput(Kbps)'] + \
                              defined_df['Video Streaming Download Throughput(Kbps)'] + \
                              defined_df['Web Page Download Throughput(Kbps)'] - \
                              defined_df['Downlink TCP Retransmission Rate(%)'] - \
                              defined_df['Video Streaming xKB Start Delay(ms)'] - \
                              defined_df['Web Average TCP RTT(ms)']
defined_df.head()

Построим гистограмму распределения этого условного параметра

In [ ]:
fig = px.histogram(defined_df, x="total_quality")
fig.update_traces(marker_line_width=1,marker_line_color="white")
fig.update_layout(title_text="Распределение параметра total_quality",
                  title_font=dict(size=22, color='blue'))
fig.show()

Как мы видим, такая гистограмма получилась довольно близкой к нормальному распределению. Используем только для этого изобретенного нами параметра нашу функцию.

In [ ]:
sf.draw_correlation_graphs(defined_df,
                           target_feature_idx = 1,
                           columns_idxs = [11],
                           meth_list = ['spearman', 'kendall'])

Видим, что и этот параметр никак не кореллирует с оценкой абонента.

Теперь давайте еще попробуем заменить размазанную шкалу из 10 оценок шкалой из трех - хорошая оценка (9 и 10), средняя оценка (от 5 до 8) и низкая оценка (менее 5). Проверим, будет ли корелляция в этом случае.

In [ ]:
def separation(x):
    if x in [9, 10]:
        return 3
    elif x in range(5, 9):
        return 2
    else:
        return 1

In [ ]:
defined_df['Q1_group'] = defined_df['Q1'].apply(separation)
defined_df.head()

In [ ]:
sf.draw_correlation_graphs(defined_df,
                           target_feature_idx = 12,
                           columns_idxs = list(range(3, 12)),
                           meth_list = ['spearman', 'kendall'])

Нет, и здесь зависимости не прослеживается. Это дает нам основание провести более тщательное исследование с помощью статистических методов.

Будем использовать наш почищенный датафрейм. Давайте посмотрим сначала на картину распределения технических параметров. (на диагонали графика)

In [ ]:
clean_df = megafon_df[~megafon_df['user_id'].isin(list(undefined_df['user_id']))]
clean_df = clean_df.astype({"Q1": "Int64"})
sns.pairplot(clean_df)

Как мы видим - действительно, все тех.параметры связи кроме Total Traffic(MB) имеют ненормальное распределение с ярко выраженным правым "хвостом". Давайте подрежем "хвосты" по квантилю 0.97 и посмотрим на новую картину распределения.

In [ ]:
q_hi_list = [clean_df[col].quantile(0.97)  for col in clean_df.columns[4:]]
for i, col in enumerate(clean_df.columns[4:]):
    clean_df = clean_df[clean_df[col] < q_hi_list[i]]
clean_df.shape

In [ ]:
sns.pairplot(clean_df)

При этом, как видно, мы теряем не так много данных, зато распределения уже гораздо ближе к нормальным. Далее будем работать с таким датафреймом.

Перед статистическими тестами давайте ещё посмотрим на картину корелляции тех.параметров связи между собой

In [ ]:
corr = clean_df[clean_df.columns[3:]].corr()
sns.set(rc = {'figure.figsize':(15,8)})
sns.heatmap(corr,
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values,
            annot = True)

Из таблицы видно, что хорошо кореллируют между собой Downlink Throughput(Kbps), Video Streaming Download Throughput(Kbps) и Web Page Download Throughput(Kbps). Это и понятно, ведь все эти параметры связаны с хорошим каналом передачи данных к абоненту. Также они имеют достаточно сильную обратную корелляцию с параметрами Video Streaming xKB Start Delay(ms) и Web Average TCP RTT(ms), что также понятно. Одно вытекает из другого.

Итак, уберем выбросы по вышеотработанной схеме из нашего основного датафрейма.

In [ ]:
q_hi_list = [defined_df[col].quantile(0.97)  for col in defined_df.columns[4:11]]
for i, col in enumerate(defined_df.columns[4:11]):
    defined_df = defined_df[defined_df[col] < q_hi_list[i]]

In [ ]:
len(defined_df)

## Пояснение

Почти все наши тех. параметры связи имеют ненормальное распределение.<br> Проверить альтернативную гипотезу можно параметрическим или непараметрическим критерием. Параметрические опираются на параметры выборочного распределения случайной величины и обладают большей мощностью (реже допускают ошибки второго рода). Из непараметрических критериев популярен критерий Манна — Уитни. Его стоит применять, если ваши выборки очень малого размера или есть большие выбросы (метод сравнивает медианы, поэтому устойчив к выбросам). Специально для Оксаны Лимановской напишу, почему далее я использовал бутстреп. <br>Во-первых, сильные выбросы я почистил. При этом, конечно, распределение не стало нормальным, но стало сильно ближе к нему.<br>Во-вторых, Манн-Уитни хорошо работает на данных с выборкой до 60 значений. У нас в любом случае гораздо больше. И вот тут как раз при достаточности выборки бутстреп даёт гораздо более достоверные результаты. Манн_Уитни всё равно будет склоняться к положительным результатам - то есть разница есть, тогда как бутстреп при достаточной выборке даст более достоверную картину. Поэтому Манна-Уитни любят те аналитики, которые больше пускают пыль в глаза заказчика, применяя его при любом размере выборки, как только видят ненормальное распределение. Он чаще всего в таком случае даст то, что заказчик лучше "съест" - да, разница есть. Хотя по факту её может и не быть. 

## Перейдём к статистическим тестам

Для проведения статичтических тестов я решил не выбирать какие-то отдельные параметры, а провести комплексное исследование. Для этого была создана функция bootstrep и помещена в скрипт с предыдущей функцией. Обе их мы импортируем в начале блокнота. У этой функции очень сложная архитектура. Давайте подробно разберем ее функционал.

На вход функция принимает:<br>- Датафрейм для исследования<br>- Индекс столбца датафрейма, статистику по которому мы хотим проверить<br>- Список индексов столбцов, статистическое сранение с которыми относительно целевой метрики, мы хотим провести<br>- Список из нулей и единиц, длина которого равна списку передаваемых столбцов для исследования, а ноль означает, что меньшие значения параметра соответствующего столбца лучше, единица - что лучше большие значения параметра<br>- Размер выборки для бутстрепа - это или конкретное целое число, или если мы передаём "-1", то размер выборки равен количеству элементов в соответствующем столбце тех.параметра связи (по умолчанию -1)<br>- Количество выборок для бутстрепа (по умолчанию 1000)<br>- Функцию для статистического сравнения - можно указать среднее (np.mean) или медианное (np.median) (по умолчанию np.mean)<br>- Метод для статистического сравнения. Может принимать значение "by_func" - так как наш тех. параметр связи непрерывен, мы не можем определить долю абонентов в тестовой и контрольной группе, выбравших оценку связи на основе какого то конкретного значения этого параметра, поэтому будем сравнивать на основе пересечения доверительных интервалов средних или медианных значений выборок из контрольной и тестовой группы (среднее или медианное определяем мы, передавая соответстующую функцию - см. предыдущий пункт) и вхождения нуля в доверительный интервал значений разницы средних или медианных выборок из контрольной и тестовой групп. Если мы передаём в качестве метода "prop" - наша функция определяет некое пороговое значение тех. параметра связи, выше которого считаем параметр хорошим, а ниже плохим. Этот порог функция рассчитывает на основе ряда процентилей - 90%, 80%, 70%, 60%, и 50%. То есть при 90% функция считает хорошим значение параметра, если он выше чем 90% значений этого параметра. И так далее. Затем при таком пороге счтиаются доверительные интервалы для доли абонентов с хорошим параметром в контрольной и тестовой группе и проверяется пересекаются они или нет. Также это позволяет нам рассчитать p_value. Далее p-value сравнивается с уровнем значимости (мы принимаем 0.05). Если он меньше уровня значимости, мы можем говорить о разнице между контрольной и тестовой группой.

Также наша функция кроме построения множества графиков и рассчета статитики еше и возвращает датафрейм. Столбцы в нём - это тех. параметры связи, переданные нами в функцию. Рассмотри строки. Наша функция помимо всего прочего разделяет на контрольную и тестовую группу на основании оценки абонента при ответе на первый вопрос (при передаче в функцию в качестве целевого аргумента для исследования столбец с этими оценками). Делит она так - сначала тестовая группа это те, кто поставил 10, контрольная - это те, кто поставил другие оценки. Далее тестовая - это 10 и 9, контрольная - остальные. И так до оценки 5, то есть крайнее разбиение - это тестовая группа 6, 7, 8, 9 ,10 и контрольная - остальные. При переданном методе "by_func" далее для каждого такого разбиения на группы считается статистическая разница между контрольной и тестовой группой на основании пересечения доверительных интервалов и вхождения нуля в доверительный интервал. На пересечении столбцов и строк мы видим значение "Да",  "Нет" или "Тест хуже!", то есть разница есть, разницы нет или разница есть, но альтернативная гипотеза, выдвинутая нами, может быть отвергнута - зато может быть принята обратная гипотеза. То есть, если мы выдвигали альтернативную гипотезу, что параметр связи для тестовой группы будет лучше, чем для контрольной, по факту оказывается, что наоборот - параметр связи для тестовой группы хуже, чем для контрольной.<br>При переданном методе "prop" после деления на группы функция также делит данные по процентилям (см. описание выше) и уже для каждого сочетания групп и процентилей рассчитывается статистическая разница по пересечению доверительных интервалов и по p-value. На пересечении строк и столбцов могут быть значения: "Да" - есть разница, "Нет"  - нет разницы.

Давайте же запустим нашу функцию сначала используя метод "by_func" (менее точный), а затем "prop" (более точный).

In [ ]:
res_df = sf.bootstrep(defined_df,
                      target_feature_idx = 1,
                      columns_idxs = [i for i in range(3, 12)],
                      hypotheses_list = [1, 1, 1, 0, 1, 0, 1, 0, 1],
                      n = -1,
                      N_TRIAL = 3000,
                      func = np.mean,
                      method = 'by_func')

In [ ]:
s = res_df.style.apply(lambda x: ["background-color:lightgreen;" if i == 'Да' \
                              else "background-color:pink;" for i in x],
                   axis=1) ## Axis=1 : Row and Axis=0: Column

s

In [ ]:
res_df = sf.bootstrep(defined_df,
                      target_feature_idx = 1,
                      columns_idxs = [i for i in range(3, 12)],
                      hypotheses_list = [1, 1, 1, 0, 1, 0, 1, 0, 1],
                      n = -1,
                      N_TRIAL = 3000,
                      func = np.mean,
                      method = 'prop')

In [ ]:
s = res_df.style.apply(lambda x: ["background-color:lightgreen;" if i == 'Да' \
                              else "background-color:pink;" for i in x],
                   axis=1) ## Axis=1 : Row and Axis=0: Column

s

А теперь глазами оцениваем все результаты и делаем выводы по оценкам, поставленным абонентами Мегафона при ответе на первый вопрос.

# Выводы по первому вопросу опроса

Рассмотрим каждый тех. параметр связи и посмотрим, влияет ли он на оценку абонентов при ответе на первый вопрос опроса.

**Примечание**: чтобы повнимательнее посмотреть на какой либо отдельный параметр, надо запустить функцию sf.bootstrep с параметром columns_idxs = [<индекс столбца нужного параметра>] и hypotheses_list = [<0 если лучше меньшее значение параметра и 1 если большее>]

Также для удобства я написал ещё одну функцию, которую можно вызвать командой sf.p_value_graph, которая для указанного параметра строит графики p-value следующим образом.

На вход функция принимает:<br>- Датафрейм для исследования<br>- Индекс столбца датафрейма, статистику по которому мы хотим проверить<br>- Индекс столбца датафрейма, статистическое сранение с которым относительно целевой метрики, мы хотим провести<br>- Ноль или единицу. Ноль означает, что меньшие значения параметра соответствующего столбца лучше, единица - что лучше большие значения параметра<br>- Размер выборки для бутстрепа - это или конкретное целое число, или если мы передаём "-1", то размер выборки равен количеству элементов в соответствующем столбце тех.параметра связи (по умолчанию -1)<br>- Количество выборок для бутстрепа (по умолчанию 1000)<br>- Функцию для статистического сравнения - можно указать среднее (np.mean) или медианное (np.median) (по умолчанию np.mean)

Функция для анализа оценок по первому вопросу строит для выбранного параметра 5 графиков, каждый из которых соответствует делению на контрольную и тестовую группы по определённой оценке. Для каждого такого деления идёт подделение на указанные группы еще и по некоему порогу значений исследуемого параметра, выше которого параметр будем считать хорошим, а ниже - плохим. Это деление идёт по квантилям значений параметра, начиная с 95%-ного квантиля и заканчивая 5%-ным квантилем. Для каждого такого порога считается p-value и строится соответствующий график. Таким образом в каждом случае разделения оценок на плохие и хорошие, для каждого порога параметра высчитывается p-value и его значения откладываются по оси y, а по оси x - квантили, по которым делим параметр на плохой и хороший.

Дополнительно на график наносим красные горизонтальные сплошные линии, соответствующие значениям уровня значимости 0.05 и 0.95, а также красная горизонтальная пунктирная линия, соответствующая значению уровня значимости 0.5.

Графики позволяют увидеть динамику p-value при изменении порога деления параметра на хороший и плохой. При этом часть графика ниже линии 0.05 будет говорить о том, что на этом отрезке значений порога доля хорошего параметра у тестовой группы с уровнем доверия 95% будет больше, чем у контрольной, то есть при уровне значимости 5% можно отвергнуть нулевую гипотезу в пользу альтернативной.

Если же график идёт выше линии 0.95, то это говорит о том, что может быть принята гипотеза, обратная альтернативной. То есть на этом отрезке значений порога доля хорошего параметра у тестовой группы с уровнем доверия 95% будет меньше, чем у контрольной.

Например, запустим эту функцию для первого параметра - Total Traffic(MB)

In [ ]:
sf.p_value_graph(defined_df,
                      target_feature_idx = 1,
                      column_idx = 3,
                      hypotheses_type = 1,
                      n = -1,
                      N_TRIAL = 3000,
                      func = np.mean)

## 1. Параметр Total Traffic(MB)

&emsp;Изначально распределение этого параметра не имеет ярко выраженных выбросов. Оно достаточно равномерное у всех абонентов.<br>&emsp;При методе "by_func" ни в одном из вариантов разбиения на контрольную и тестовую группу по оценкам разницы между группами по этому параметру нет на уровне доверия 95%.<br>&emsp;При методе "prop" опять же нет разницы при любом делении на группы и при любом пороге деления параметра на хороший и плохой.

&emsp;А вот наша функция по построению графиков p-value даёт нам больше информации. При делении параметра на хороший и плохой по высокому порогу, p-value ближе к нижнему уровню значимости, то есть ближе к возможности отвергнуть нулевую гипотезу в пользу альтернативной. То есть абоненты с высоким уровнем траффика ближе к тому, чтобы поставить хорошую оценку. Причем наиболее ярко эта тенденция выражена, если считать хорошей оценку более 5.

&emsp;При уменьшении порога хорошего параметра есть четкая тенденция к смещению p-value в стороны гипотезы, обратной альтернативной и если считать хорошим параметр более 55%-ного квантиля, то в некоторых случаях появляются основания отвергнуть нулевую гипотезу в пользу обратной альтернативной. То есть если считать хорошим трафик со значениями больше 55%-ного квантиля, то абоненты, значение траффика у которых близко к этому порогу или ниже более склонны поставить хорошую оценку. При этом наиболее ярко эта тенденция проявляется, если считать хорошей оценку от 7 до 9 включительно.

&emsp;**Параметр на уровне доверия 95% не влияет на поставленную оценку. Тем не менее есть тенденция: при высоком пороге деления траффика на хороший и плохой доля абонентов с хорошим траффиком, поставивших высокую оценку, выше. То есть при высоких значениях траффика абоненты "чувствуют" его изменение в худшую сторону и ставят соответствующую оценку. При более низких значениях порога деления параметра на хороший и плохой в тестовой группе растет количество абонентов с более низким траффиком. И это ведет к смещению p-value в сторону гипотезы, обратной альтернативной. Примерно при 60-70%-ном пороге абоненты перестают ощущать разницу в изменении траффика в ту или другую сторону. А при еще более низком пороге абоненты всё отчётливее начинают реагировать на низкие значения траффика, при этом ставя хорошую оценку при более низких его значениях. То есть доля абонентов с более низким траффиком, принятым нами плохим, поставивших при этом хорошую оценку, растёт.**

&emsp;**Чем это вызвано? Я считаю, что абонентов можно разделить на активно использующих траффик - это люди, использующие мобильный интернет для просмотра видео, в том числе на телевизоре через мегафон модем, и использующих менее активно - это люди, использующие мобильный интернет в основном для сёрфинга по соц сетям и для интернет-звонков. Поэтому при высоких значениях траффика мы имеем дело с первой группой. Они сильно чувствуют разницу в скорости интернета, так как видео, особенно высокого разрешения, крайне требовательно к качеству и сорости интернета. При средних и более низких значениях траффика доля абонентов из второй группы увеличивается и они ставят более низкую оценку несмотря на больший объем траффика. По идее p-value при этом должно было бы стабилизироваться, но тем не менее оно растёт. То есть траффик-то перестаёт влиять на оценку, потому что это нелогично - ставить хорошую оценку при более плохом траффике, видимо в этом случае на оценку начинают влиять какие-то другие факторы и несмотря на то что в одной группе траффик будет лучше, чем в другой, оценку они поставят хуже. Значит она обусловлено влиянием чего то другого и это что-то тем не менее как то связано с траффиком. То есть при большем относительно некоего порога (судя по графикам начинающегося в районе 55%-ного квантиля и ниже) траффике есть бОльшая вероятность возникновения каких-то технических проблем, котрые ведут к тому, что абоненты ставят несмотря на лучший траффик более низкую оценку.**

&emsp;**Резюмируя, можно сказать что траффик с уровнем доверия 95% слабо влияет на поставленную оценку.**

In [ ]:
sf.p_value_graph(defined_df,
                      target_feature_idx = 1,
                      column_idx = 4,
                      hypotheses_type = 1,
                      n = -1,
                      N_TRIAL = 3000,
                      func = np.mean)

## 2. Параметр Downlink Throughput(Kbps)

&emsp;При методе "by_func" при любом раскладе есть статистическая разница между группами<br>&emsp;При методе "prop" также при любом раскладе разница есть, если судить по p-value. Если же смотреть по пересечению доверительных интервалов, то картина такая: при четко выраженном делении на контрольную и тестовую группу по оценке 10 (то есть тестовая группа - поставившие 10, контрольная - все остальные) разница есть всегда. А вот при остальных вариантах деления и при ярко выраженном делении параметра на хороший и плохой (по квантилям 90% и 80%) интервалы могут пересекться краешками и не показывать разницы. Тем не менее p-value более точный параметр, а пересечение интервалов помогает нам просто увидеть динамику стремления разницы между группами к большему или меньшему значению.

&emsp;**То есть можно сказать, что параметр Downlink Throughput(Kbps) с уровнем доверия 95% (и даже в большинстве случаев с уровнем доверия 99%) влияет на оценку абонентов при ответе на первый вопрос. Чем больше этот параметр, тем выше оценка.**

In [ ]:
sf.p_value_graph(defined_df,
                      target_feature_idx = 1,
                      column_idx = 5,
                      hypotheses_type = 1,
                      n = -1,
                      N_TRIAL = 3000,
                      func = np.mean)

## 3. Параметр Uplink Throughput(Kbps)

&emsp;В большинстве случаев разницы между контрольной и тестовой группой нет при уровне доверия 95%.

&emsp;**При любых раскладах по обоим методам никакой разницы между тестовой и контрольной группой практически нет. Параметр никак не влияет на поставленную оценку. Тем не менее график достаточно близок к уровню значимости, но тем не менее не пересекает его, кроме одного случая - деление на группы по оценке 8 и деление на хороший и плохой параметр по квантилю 90%.**

In [ ]:
sf.p_value_graph(defined_df,
                      target_feature_idx = 1,
                      column_idx = 6,
                      hypotheses_type = 0,
                      n = -1,
                      N_TRIAL = 3000,
                      func = np.mean)

## 4. Параметр Downlink TCP Retransmission Rate(%)

&emsp;Не забываем, что при анализе графиков p-value для этого параметра рассматриваем график слева направо, так как здесь лучше меньшие значения параметра.

&emsp;При методе "by_func" при любом делении на контрольную и тестовую группу кроме варианта деления "тест - 10, контроль - все остальные" есть статистическая разница между группами. Это говорит о том, что абоненты не "чувствуют" разницы в значении параметра при оценке 9 и при оценке 10.

&emsp;При методе "prop" зависимость сохраняется.

&emsp;**Параметр влияет на оценку, причем видно, что чем выше порог деления на хорошую и плохую оценку, тем сильнее ощущается разница между контрольной и тестовой группой при любом значении порога деления параметра на хороший и плохой. Также разница стремится к статистически неразличимой при значении порога деления параметра на хороший и плохой около минимальных значений (менее 15%-ного квантиля). Всё это говорит о том, что абоненты ощущают разницу в значениях этого параметра, но при этом не так сильно, как например параметра Downlink Throughput(Kbps). Поэтому им тяжелее дифференцировать оценку этого параметра на 10 разных оценок. У тех кто ставит 9 и 10 разницы по параметру практически нет. То есть выбор, что поставить - 9 или 10 - определяется не этим параметром. Также при делении на хорошую/плохую оценку по оценке 8 абоненты не ощущают разницы при значениях параметра, близким к минимальным, то есть если считать хорошим значение параметра менее 15%-ного квантиля, то оценка не будет от него зависеть. То есть изменение параметра в пределах от 0 до 15%-ного квантиля для абонентов неразличимо. Можно сказать, что значение параметра, равное примерно 15%-ному квантилю, уже достаточно хорошее и дальнейшее его уменьшение уже не ощутимо абонентами.**

In [ ]:
sf.p_value_graph(defined_df,
                      target_feature_idx = 1,
                      column_idx = 7,
                      hypotheses_type = 1,
                      n = -1,
                      N_TRIAL = 3000,
                      func = np.mean)

## 5. Параметр Video Streaming Download Throughput(Kbps)

&emsp;При методе "by_func" при любом делении на контрольную и тестовую группу есть разница между вариантами, причем судя по графикам разница более ощутима при большем обобщении хороших оценок, то есть чем ниже взять границу хороших и плохих оценок, тем разница становится ощутимее.

&emsp;При методе "prop" динамика сохраняется.

&emsp;**Так как этот параметр хорошо кореллирует с параметром Downlink Throughput(Kbps), то он показывает сходный характер поведения. При этом по графикам p-valye можно увидеть, что при значении порога деления значений параметра на хороший и плохой, равног 85%-ному квантилю и выше, абоненты поставившие больше 7 и меньше 7 не различают при уровне доверия 95% разницы между хорошим и плохим параметром. То есть для абонентов, поставивших более 7, достаточно значение параметра, равное 85%-ному квантилю. Дальнейшее увеличение параметра ими уже не ощущается.**

In [ ]:
sf.p_value_graph(defined_df,
                      target_feature_idx = 1,
                      column_idx = 8,
                      hypotheses_type = 0,
                      n = -1,
                      N_TRIAL = 3000,
                      func = np.mean)

## 6. Параметр Video Streaming xKB Start Delay(ms)

&emsp;Не забываем, что при анализе графиков p-value для этого параметра рассматриваем график слева направо, так как здесь лучше меньшие значения параметра.

&emsp;При методе "by_func" при любом делении на контрольную и тестовую группу есть разница между вариантами, причем судя по графикам разница более ощутима при большем обобщении хороших оценок и в принципе, если брать порог хорошей оценки 5, 6 или 7, эта разница уже не меняется и остается максимальной.

&emsp;При методе "prop" зависимость по делению на группы сохраняется.

&emsp;**Параметр влияет на оценку при любом раскладе при уровне доверия 95% (в большинстве случаев даже при уровне доверия 99%)**

In [ ]:
sf.p_value_graph(defined_df,
                      target_feature_idx = 1,
                      column_idx = 9,
                      hypotheses_type = 1,
                      n = -1,
                      N_TRIAL = 3000,
                      func = np.mean)

## 7. Параметр Web Page Download Throughput(Kbps)

&emsp;При методе "by_func" при любом делении на контрольную и тестовую группу есть разница между вариантами, причем судя по графикам разница более ощутима при большем обобщении хороших оценок и максимальна при делении оценок на хорошие и плохие по порогу 5.

&emsp;При методе "prop" мы видим, что при делении на тестовую и контрольную группу по порогам оценок больше 5 и при высоком пороге деления параметра на хороший и плохой нет оснований отвергнуть нулевую гипотезу. Разница между гркуппами отсутствует. В остальных случаях при уровне доверия 95% можно сказать, что разница существует

&emsp;**Параметр в большинстве случаев влияет на поставленную оценку. Но есть исключения. Вспомним наше предположение о делении абонентов на две группы. При высоких оценках в тестовой группе присутствует много абонентов, использующих интернет для видео и им не так важна скорость загрузки веб страниц, поэтому при высоких порогах деления параметра на хороший и плохой нет статистической разницы между группами. При понижении порога хорошей оценки разница при уровне доверия 95% начинает появляться при всё более высоких порогах хорошего параметра. Пока, наконец, при делении оценок на хорошую и плохую по порогу 5 при любых значениях параметра разница имеет место быть всегда. Это объясняется тем, что при более низких значениях параметра в тестовой группе появляется всё больше абонентов, использующих интернет для серфинга по соц сетям и интернет-звонков и им загрузка веб-страниц важна и низкое значение параметра начинает ими замечаться.**

In [ ]:
sf.p_value_graph(defined_df,
                      target_feature_idx = 1,
                      column_idx = 10,
                      hypotheses_type = 0,
                      n = -1,
                      N_TRIAL = 3000,
                      func = np.mean)

## 8. Параметр Web Average TCP RTT(ms)

&emsp;Не забываем, что при анализе графиков p-value для этого параметра рассматриваем график слева направо, так как здесь лучше меньшие значения параметра.

&emsp;При методе "by_func" при любом делении на контрольную и тестовую группу есть разница между вариантами, причем судя по графикам разница более ощутима при большем обобщении хороших оценок и максимальна при делении оценок на хорошие и плохие по порогу 5.

&emsp;При методе "prop" зависимость по делению на группы сохраняется..

&emsp;**Параметр при уровне доверия 95% в подавляющем большинстве случаев влияет на оценку. Исключение есть только при делении на хорошую и плохую оценку по порогу 9 и делении параметра на хороший и плохой по уровню 15%-ного квантиля и ниже. Опять же это обусловлено в этом случае наличием в тестовой группе большого количества абонентов, использующих интернет для видео. Им не важен этот параметр. При более высоких его значениях свой вклад начинают вносить абоненты, использующие интернет для соц сетей и интернет-звонков, их уже начинает не устраивать такой пинг и они ставят худшую оценку.**

# Общий вывод по первому вопросу для бизнеса

&emsp;1. Параметр Total Traffic(MB) сам по себе не влияет на оценку. Он скорее является следствием качества других технических параметров. Тем не менее он даёт повод обратить на себя внимание. У абонентов с высоким траффиком больше склонность к выставлению высокой оценки по первому вопросу. У абонентов с более низким траффиком появляется следующая тенденция: если их поделить по какому то значению траффика на две группы, то лучшую оценку склонны поставить абоненты в группе с худшим траффиком. Видимо качество каких то других параметров приводит к такому несоответствию. Возможно ответ на второй вопрос нам поможет.

&emsp;Также забегая вперёд сделаем предположение, что на такую картину влияет распределение абонентов на две группы - использующие мобильный интернет в основном для видео и использующие его в основном для серфинга по соцсетям и интернет-звонков. Технические параметры связи влияют на оценку абонентов из этих рупп по разному. Поэтому исследуя их совместно, при каких то уровнях параметров сильнее начинает влиять на оценку то одна, то другая группа. И совместно они "наводят помехи" в анализе параметров друг на друга. Я бы предложил в опросе добавить еще один вопрос - "Для чего в основном вы используете мобильный интернет?". И три варианта ответа - "Для просмотра видео, в том числе на смарт тв приставках", "Для соц.сетей, мессенджеров и интернет-звонков" и "И для того, и для другого в равной степени". А затем при статистическом анализе рассматривал бы каждую группу отдельно.

&emsp;2. Не влияет на оценку только параметр Uplink Throughput(Kbps) - это единственный параметр "от абонента" и его влияние на общее качество для абонентов незаметно.

&emsp;2. Все остальные параметры связи, "к абоненту", влияют на оценку. Для параметра Downlink Throughput(Kbps) это вилияние ощущается сильнее всего. Абоненты обоих вышеописанных групп "чувствуют" его изменение и шкала в 10 оценок имеет смысл, так как разница по этому параметру между поставившими каждый из вариантов оценок есть. О других параметрах так сказать нельзя, у них "чувствительность" поменьше. Между абонентами, поставившими близкие оценки, разница размыта. Тем не менее можно говорить о том, что ещё не достигнут верхний предел значений технических параметров связи, при котором абоненты перестанут ощущать разницу. Значит надо улучшать все параметры путём модернизации оборудования.